# Use panaroo's output file (final_graph.gml) to generate a Node specific file and an Edge specific file

In [2]:
!pip install pandas
!pip install networkx

     |████████████████████████████████| 1.6 MB 7.1 MB/s eta 0:00:01


In [1]:
import networkx as nx
import pandas as pd

In [2]:
#Read the graph
G_id = nx.read_gml('final_graph.gml',label='id')

In [3]:
#Convert graph with id as labels to dataframe
dicgradic = dict(G_id.nodes(data=True))
base_df = pd.DataFrame.from_dict(dicgradic, orient='index')

In [4]:
reindex_base = base_df.reset_index()

In [5]:
#Convert edge data to dataframe
edge_df = nx.to_pandas_edgelist(G_id)

In [10]:
edge_df.head()

,source,target,members,genomeIDs,size
0,0,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...,47
1,0,3240,"[1, 2, 3, 8, 9, 12, 14, 15, 19, 23, 31, 33, 34...",1;2;3;8;9;12;14;15;19;23;31;33;34;35;39;40;41;...,20
2,1,8431,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...,47
3,2,3,"[0, 1, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",0;1;3;4;6;7;8;9;10;11;12;13;14;16;17;19;21;22;...,42
4,2,4071,[5],5,1


In [15]:
#Remove special characters for ease of readability for Neo4j
reindex_base["members"]=reindex_base["members"].apply(lambda x: str(x).replace("[", "").replace("]","").replace(",","~").replace(" ",""))
reindex_base["seqIDs"]=reindex_base["seqIDs"].apply(lambda x: str(x).replace("[", "").replace("]","").replace(",","~").replace("'","").replace(" ",""))
reindex_base["lengths"]=reindex_base["lengths"].apply(lambda x: str(x).replace("[", "").replace("]","").replace(",","~").replace(" ",""))
edge_df["members"]=edge_df["members"].apply(lambda x: str(x).replace("[", "").replace("]","").replace(",","~").replace(" ",""))

In [16]:
edge_df.head(2)

,source,target,members,genomeIDs,size
0,0,1,0~1~2~3~4~5~6~7~8~9~10~11~12~13~14~15~16~17~18...,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...,47
1,0,3240,1~2~3~8~9~12~14~15~19~23~31~33~34~35~39~40~41~...,1;2;3;8;9;12;14;15;19;23;31;33;34;35;39;40;41;...,20


In [17]:
reindex_base.head(2)

,id,label,size,centroid,maxLenId,members,seqIDs,hasEnd,protein,dna,annotation,description,lengths,longCentroidID,paralog,mergedDNA,genomeIDs,geneIDs,degrees,name
0,0,2624,47,1_0_3237,0,0~1~2~3~4~5~6~7~8~9~10~11~12~13~14~15~16~17~18...,24_0_0~6_0_0~9_0_3124~26_0_0~25_0_0~2_0_2739~1...,1,VSVELWQQCVDLLRDELPSQQFNTWIRPLQVEAEGDELRVYAPNRF...,GTGTCCGTGGAACTTTGGCAGCAGTGCGTGGATCTTCTCCGCGATG...,dnaA,Chromosomal replication initiator protein DnaA,1545~1545~1545~1545~1545~1545~1545~1545~1545~1...,"[1545, 1_0_3237]",0,0,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...,24_0_0;6_0_0;9_0_3124;26_0_0;25_0_0;2_0_2739;1...,2,dnaA
1,1,5572,47,0_0_1,0,0~1~2~3~4~5~6~7~8~9~10~11~12~13~14~15~16~17~18...,13_0_1~16_0_1~2_0_2738~21_0_1~41_0_6113~35_0_5...,0,MHFTIQREALLKPLQLVAGVVERRQTLPVLSNVLLVVEGQQLSLTG...,ATGCATTTCACCATTCAACGCGAAGCCCTGTTGAAACCGCTGCAAC...,dnaN,Beta sliding clamp,1104~1104~1104~1104~1104~1104~1104~1104~1104~1...,"[1104, 0_0_1]",0,0,0;1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18...,13_0_1;16_0_1;2_0_2738;21_0_1;41_0_6113;35_0_5...,2,dnaN


In [13]:
reindex_base=reindex_base.rename(columns={"index": "id"})

In [18]:
#Save dataframes separately
reindex_base[['id','name','label','description','seqIDs','lengths','annotation','dna','protein']].to_csv("Nodes_base.csv",index=None)
edge_df[['source','target','size','members']].to_csv("Edges_base.csv",index=None)